Install needed libriaries 

In [118]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Import more libraries (from previous webscrapping practice, some redundant libraries)

In [119]:
from bs4 import BeautifulSoup as Soup
import requests
from pandas import DataFrame
import seaborn as sns
import matplotlib.pyplot as plt
from os import path
import numpy as np

request html info for https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

#### Webscrapping Toronto Postal Codes with BeautifulSoup
Pull in webpage html and create tables list

In [198]:
table = pd.read_html('https://web.archive.org/web/20181110230135/https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

Assign parsed rows to a pandas dataframe

In [152]:
df = DataFrame(table[2])
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Assign Headers Manually

In [153]:
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Drop columns with Borough = 'Not Assigned'

In [154]:
df = df[df.Borough != 'Not assigned']
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


GroupBy PostalCode and Borough, and have the Neighborhoods in each Borough separated with a comma and space when there are mulitple

In [155]:
df = df.groupby(['PostalCode', 'Borough'], as_index = False).aggregate(lambda x: ', '.join(x))
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Clean up the Neighborhood Column by replacing 'None' with borough name and then removing None, form Neighborhood column


In [161]:
df['Neighborhood'] = df['Neighborhood'].replace('Not assigned', df['Borough'])
print(df.loc[df['Neighborhood'] == df['Borough']])
df.head()

   PostalCode       Borough  Neighborhood
85        M7A  Queen's Park  Queen's Park


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Get the shape of the new dataset

In [162]:
df.shape

(103, 3)

### Latitude and Longitude from Geocoder

I had difficulty with the geocoder so I ended up downloading the csv file

In [174]:
latlong = pd.read_csv('C:/Users/Tim/Desktop/CapstoneProject/Geospatial_Coordinates.csv')
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [177]:
latlong.shape

(103, 3)

pd.merge(df, latlong, left_on = 'PostalCode', right_on = 'Postal Code').head()

In [189]:
df = pd.merge(df, latlong, left_on = 'PostalCode', right_on = 'Postal Code')
#df = df[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
df.head()

,PostalCode,Borough,Neighborhood,Postal Code_x,Latitude_x,Longitude_x,Postal Code_y,Latitude_y,Longitude_y,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353,M1B,43.806686,-79.194353,M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497,M1C,43.784535,-79.160497,M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711,M1E,43.763573,-79.188711,M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917,M1G,43.770992,-79.216917,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476,M1H,43.773136,-79.239476,M1H,43.773136,-79.239476


In [192]:
df = df[['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
